In [3]:
import numpy as np 
import cv2
import matplotlib
from matplotlib import pyplot as plt 
import torch
import pywt
from PIL import Image

Testing

In [4]:
face_cascade = cv2.CascadeClassifier('./opencv/haarcascades/haarcascade_frontalface_default.xml')
device='cuda'
model=torch.load('models/entire_model_wavelet')
model.eval()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU()
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): ReLU()
  (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (16): Dropout(p=

In [5]:
def get_cropped_face(image):
    if type(image)==str:
        image = cv2.imread(image)
    if image is not None:
        gray_img=cv2.cvtColor( image,cv2.COLOR_RGB2GRAY )
        faces = face_cascade.detectMultiScale(gray_img, 1.4, 5)
        if  faces.any():
            for (x,y,w,h) in faces:
                roi_gray = gray_img[y:y+h, x:x+w]
                return roi_gray
  

In [6]:
def w2d(img, mode='db1', level=5):
    imArray = img
    #Datatype conversions
    #convert to grayscale
    # imArray = cv2.cvtColor( imArray,cv2.COLOR_RGB2GRAY )
    #convert to float
    imArray =  np.float32(imArray)   
    imArray /= 255;
    # compute coefficients 
    coeffs=pywt.wavedec2(imArray, mode, level=level)

    #Process Coefficients
    coeffs_H=list(coeffs)  
    coeffs_H[0] *= 0;  

    # reconstruction
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H =  np.uint8(imArray_H)

    return imArray_H

In [7]:
def predict_age(image_path):
    crop_img=get_cropped_face(image_path)
    if crop_img.any():
        res_img=cv2.resize(crop_img,(50,50))
        wt_img=w2d(res_img)
        inp=torch.tensor(wt_img.reshape(1,50,50),dtype=torch.float32).to(device)
        op=model(inp.unsqueeze(0))
        return torch.argmax(op)

In [8]:
predict_age('datasets/roit.webp')

tensor(38, device='cuda:0')

In [9]:
predict_age('datasets/gill.webp')

tensor(26, device='cuda:0')

In [10]:
predict_age('datasets/ronaldo.webp')

tensor(37, device='cuda:0')

In [80]:
predict_age('datasets/old.jpg')

tensor(85, device='cuda:0')

In [107]:
predict_age('datasets/kid.webp')

tensor(3, device='cuda:0')